# Model evaluation using Sidekick

Use the Deployment API of the Peltarion platform via Sidekick to get predictions on samples and evaluate the performance of the deployed model in more detail.

**Note**: This notebook requires installation of Sidekick. To install the package within the notebook, run the following code:

import sys !{sys.executable} -m pip install git+https://github.com/Peltarion/sidekick#egg=sidekick

For more information about Sidekick, see: https://github.com/Peltarion/sidekick

In [ ]:
from glob import glob
import itertools
import os
import operator
import resource

from IPython.display import display, Image
import pandas as pd
from PIL import Image
from tqdm import tqdm
import sidekick
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Setup

### Paths

In [ ]:
#preprocessed dataset
dataset_path = './fruits-360/Test'

### Client to deployment

In [ ]:
#Update the URL and token
deployment_url = 'https://...'
deployment_token = '...'

client = sidekick.Deployment(
    url=deployment_url,
    token=deployment_token
)

### Helper functions

In [ ]:
def get_max_score(pred):
    max_key = 'None'
    max_score = 0
    dict = pred['fruit_class'].items()
    for key,score in dict:
        if score >= max_score:        
            max_key = key
            max_score = score
    return (max_key, max_score)


def get_image(path):
    im = Image.open(path)
    new_im = im.copy()
    new_im.format = 'jpeg'
    im.close()
    return new_im

### Progress bars for pandas

In [ ]:
tqdm.pandas()

## Create test dataframe - ground truth

In [ ]:
images_rel_path = glob(dataset_path + '/*/*.jpg') + glob(dataset_path + '/*/*.png')
print("Images found: ", len(images_rel_path))

In [ ]:
df = pd.DataFrame({'image': images_rel_path})
df['fruit_class'] = df['image'].progress_apply(lambda path: os.path.basename(os.path.dirname(path)))
df = df.sample(frac=1).reset_index(drop=True)
df.head()

## Single  predictions

In [ ]:
im_path_list = iter(list(df['image']))

In [ ]:
im_path = next(im_path_list)
im = Image.open(im_path)
display(im)
pred = client.predict(image=im)
print(get_max_score(pred))

## Multiple predictions

In [ ]:
rows = df.sample(n=10)
for i, row in rows.iterrows():
    im = Image.open(row['image'])    
    display(im)
    pred = client.predict(image=im)
    print('Ground truth: {}\nPrediction: {}'.format(row['fruit_class'], get_max_score(pred)[0]))

## Worst misclassified

Copy the image URLs to a new column.
Create a new column that contains the actual images.

In [ ]:
eval_df = df.copy()
eval_df['image_url'] = eval_df['image']
eval_df['image'] = eval_df['image'].progress_apply(lambda path: get_image(path))
predictions = client.predict_lazy(eval_df.to_dict('record'))

Get all the predictions and store these in a new column.

**Note**: This may take several minutes

In [ ]:
pbar = tqdm(total=len(eval_df))
preds = []
for p in predictions:
    preds.append(p)
    pbar.update(1)
pbar.close()

In [ ]:
eval_df['pred'] = [p['fruit_class'] for p in preds]
eval_df.head(3)

Create new columns that contain the name of highest scoring class and the score.

In [ ]:
dicts = eval_df['pred']
max_keys = []
max_scores = []

for i in dicts:
    max_val = max(i.items(), key=lambda k: k[1])     
    max_keys.append(max_val[0])
    max_scores.append(max_val[1])
eval_df['pred_class'] = max_keys
eval_df['pred_score'] = max_scores
eval_df.head(3)

Get the top-hundred samples with the highest prediction score and incorrect label, i.e., the worst misclasified samples.

In [ ]:
wrong_df = eval_df.loc[eval_df['fruit_class'] != eval_df['pred_class']]
wrong_df = wrong_df.sort_values(by=['pred_score'], ascending=False)
first_rows = wrong_df.head(100)
for i, row in first_rows.iterrows():
    display(row['image'])
    print('Ground truth: {}, Prediction: {}, Score: {}'.format(row['fruit_class'], row['pred_class'], row['pred_score']))

### Number of miclassified Apple Granny Smith

In [ ]:
df_granny_smith = wrong_df.loc[df['fruit_class'] == 'Apple Granny Smith']
print('Number of misclasified Apple Granny Smith: {}'.format(df_granny_smith.shape[0]))

### View all misclassified samples in class Apple Granny Smith

In [ ]:
for i, row in df_granny_smith.iterrows():
    im = row['image']
    display(im)

## Classification report

In [ ]:
print(classification_report(y_pred=eval_df['pred_class'], y_true=eval_df['fruit_class']))

## Overall accuracy

In [ ]:
1 - (wrong_df.shape[0] / df.shape[0])